This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [123]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [124]:
# select model to train

model_inventory = {'mktprob': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSig},
                   'mktprob_soft': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSoft},
                    'mktprob_MLR': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': MLR},
                   'AlunOwen_v0': {'X_columns': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v1': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v2': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinDropReluLinSoft},
                   'AlunOwen_v3': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR}
                               }

model = 'mktprob_MLR'
X_columns = model_inventory[model]['X_columns']
learning_rate = model_inventory[model]['learning_rate']
epochs = model_inventory[model]['epochs']
vacant_stall_indicator = model_inventory[model]['vacant_stall_indicator']
bias = model_inventory[model]['bias']
model_architecture = model_inventory[model]['model_architecture']

In [125]:
# read in data

y_columns = ["win"]

train_data_fn = "data\\runners_train.csv"
test_data_fn = "data\\runners_test.csv"

train_data = RacesDataset(train_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator)
test_data = RacesDataset(test_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [126]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

mkt_prob                                                    \
stall_number        1         2         3         4         5         6    
race_id                                                                    
11504         0.100000  0.038462  0.125000  0.058824  0.142857  0.053763   
11505         0.019608  0.029412  0.142857  0.038462  0.400000  0.058824   
11506         0.266667  0.038462  0.014925  0.058824  0.181818  0.038462   
11507         0.076923  0.058824  0.019608  0.222222  0.444444  0.029412   
11508         0.038462  0.038462  0.133333  0.047619  0.230947  0.200000   

                                                                               \
stall_number        7         8         9         10        11   12   13   14   
race_id                                                                         
11504         0.090909  0.125000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
11505         0.053763  0.142857  0.029412  0.222222  0.153846  0.0  0.0  0.0   
11506         0.111111  0.029412  0.111111  0.307692  0.000000  0.0  0.0  0.0   
11507         0.125000  0.153846  0.066667  0.000000  0.000000  0.0  0.0  0.0   
11508         0.058824  0.181818  0.250000  0.000000  0.000000  0.0  0.0  0.0   

                        
stall_number   15   16  
race_id                 
11504         0.0  0.0  
11505         0.0  0.0  
11506         0.0  0.0  
11507         0.0  0.0  
11508         0.0  0.0

In [127]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]

# torch.manual_seed(0)
net = model_architecture(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: mktprob_MLR
Layer: weights | Size: torch.Size([1]) | Values : Parameter containing:
tensor([-1.6077], requires_grad=True) 



In [128]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = net(X)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([11])
torch.Size([1, 16])


In [129]:
%env WANDB_NOTEBOOK_NAME 'C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "model": model,
    "X_columns": X_columns,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_stall_indicator": vacant_stall_indicator,
    "bias": bias,
    "model_architecture": list(net.modules())
    }
)

env: WANDB_NOTEBOOK_NAME='C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'


In [130]:
# optimizing model parameters

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, net, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

Epoch 1
-------------------------------
loss: 2.809737  [   64/16620]
loss: 2.678330  [ 6464/16620]
loss: 2.559772  [12864/16620]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.531635 

Epoch 2
-------------------------------
loss: 2.655037  [   64/16620]
loss: 2.580183  [ 6464/16620]
loss: 2.557407  [12864/16620]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.529647 

Epoch 3
-------------------------------
loss: 2.654825  [   64/16620]
loss: 2.578974  [ 6464/16620]
loss: 2.557304  [12864/16620]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.529152 

Epoch 4
-------------------------------
loss: 2.654860  [   64/16620]
loss: 2.578566  [ 6464/16620]
loss: 2.557343  [12864/16620]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.528930 

Epoch 5
-------------------------------
loss: 2.654915  [   64/16620]
loss: 2.578371  [ 6464/16620]
loss: 2.557398  [12864/16620]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.528805 

Epoch 6
-------------------------------
loss: 2.654966  [   64/16620]
loss: 2.578260  [ 64

In [131]:
# finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.35149
loss,2.52848


In [132]:
for para_name, para_vals in net.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")